In [1]:
import numpy as np
import re
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait

Collect the parameters of sub-category URL

In [ ]:
url_category_list = ["https://www.hktvmall.com/hktv/en/personalcarenhealth", "https://www.hktvmall.com/hktv/en/beautynhealth"]
url_code_list = []
url_title_list = []
url_category_list1 = []
url_category_list2 = []

for url_category in url_category_list:
    res = requests.get(url_category)
    soup = BeautifulSoup(res.text, "html.parser")

    for url_code in soup.find_all("div", class_="subnav")[0].ul.find_all("a", class_="link"):
        if url_code["data-cat"].endswith("00000000"):
            try:
                url_title_temp = url_code.text.split("&nbsp")[1].replace(",","").replace("&","and").replace(" ","-")
            except:
                continue
        else:
            url_subtitle_temp = url_code.text.split("\xa0")[0].replace(",","").replace("&","and").replace(" ","-")
            url_title_list.append(url_title_temp+"/"+url_subtitle_temp)
            url_code_list.append(url_code["data-cat"])
            if "personal" in url_category:
                url_category_list1.append("Personal-Care-and-Health")
                url_category_list2.append("personalcarenhealth")
            elif "beauty" in url_category:
                url_category_list1.append("Skincare-and-Makeup")
                url_category_list2.append("beautynhealth")
            else:
                print("Check the updated structure !!!")

Collect the post URL

In [ ]:
post_url_list = []
post_category_list = []
post_sub_category_list =[]
post_page_list = []

driver = webdriver.Chrome(service=Service('./chromedriver.exe'))
driver.set_window_position(-1000, 0)
driver.maximize_window()


if len(url_code_list) == len(url_title_list) == len(url_category_list1) == len(url_category_list2):
    for i in range(len(url_code_list)):
        for page in range(3):
            try:
                url = f"https://www.hktvmall.com/hktv/en/{url_category_list1[i]}/{url_category_list1[i]}/{url_title_list[i]}/main/search?page={page}&q=%3Arelevance%3Acategory%3A{url_code_list[i]}%3Azone%3A{url_category_list2[i]}%3Astreet%3Amain%3A"
                web_status = 0
                while web_status == 0:
                    driver.get(url)
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    if soup.img["src"] != 'error.jpg':
                        while "All Rights Reserved" not in soup.prettify():
                            WebDriverWait(driver, 5)
                            soup = BeautifulSoup(driver.page_source, "html.parser")
                        post_url_list_temp = [j["data-id"] for j in soup.find_all("div", class_="product-brief")]
                        if len(post_url_list_temp)!=0:
                            post_url_list.extend(post_url_list_temp)
                            for count in range(len(post_url_list_temp)):
                                post_category_list.append(url_category_list1[i].replace("-", " "))
                                post_sub_category_list.append(url_title_list[i].replace("-", " ").replace("/", " - "))
                                post_page_list.append(page)
                            web_status +=1
            except:
                continue
    driver.quit()
else:
    print("Check the parameter links of URL !!!")

Collect each post data

In [ ]:
common_unit_list = ["mg", "g", "kg", "ml", "l", "oz", "mm", "cm", "bag", "bottle", "box", "cap", "count", "day", "ea", "pack", "pc", "piece", "sachet", "set", "sheet", "stick", "tablet", "unit", "x"]
common_unit_list_zn = ["毫克", "克", "千克", "毫升", "升", "安士", "毫米", "厘米", "袋", "瓶", "盒", "粒", "次", "天", "個", "包", "塊", "件", "袋", "組", "張", "支", "片", "單位", "項"]

driver = webdriver.Chrome(service=Service('./chromedriver.exe'))
driver.set_window_position(-1000, 0)
driver.maximize_window()

for num in range(len(post_url_list)):
    url = f"https://www.hktvmall.com/hktv/en/p/{post_url_list[num]}"
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, "html.parser")

    ## Part 1)
    category = post_sub_category_list[num]
    page_position = post_page_list[num]
    brand_name = soup.find("h1", class_="last").text.split(" - ")[0]
    product_name = soup.find("h1", class_="last").text.split(" - ")[1]

    ## Part 2)
    for i in soup.find("script", type="application/ld+json").text.split("\n"):
        if "image" in i:
            slider_img = re.findall("\[\S+\]", i)[0].replace("[","").replace("]","").replace('''"''',"").split(",")
            slider_img_list = [j for j in slider_img if j.endswith("96.jpg") | j.endswith("96.png")]

    slider_video_list = []
    for i in soup.find("div", class_="product-img-slider").script.text.split("\n"):
        if ("video" in i) and len(re.findall("\[\S+\]", i))!=0:
            slider_video = re.findall("\[\S+\]", i)[0].replace("[","").replace("]","").replace('''"''',"")
            slider_video_list.append(slider_video)
    slider_video_list = list(set(slider_video_list))
    
    ## Part 3)
    # Example reason for no scraping "promotional" class - https://www.hktvmall.com/hktv/en/p/H8912001_S_HH_8806334371807_01
    product_price = float(soup.find("div", class_="price").text.replace("$",""))
    if "merchant" not in soup.find("div", class_="deliveryLabelText").text:
        delivery = soup.find("div", class_="deliveryLabelText").text.replace("-Day Delivery", " day(s)")
    else:
        delivery_temp = soup.find("tr", class_="product-return-delivery").li.text.replace(" ", "").replace("\r", "").replace("\n", "").replace("Days", " day(s)")
        delivery = delivery_temp[re.search("\d", delivery_temp).start():]
    try:
        sales_count_over = int(re.search("\d+\+", soup.find("div", class_="salesNumber-container").span.text.replace(",", "")).group()[:-1])
    except:
        continue

    packing_spec_number_list = []
    packing_spec_unit_list = []
    packing_spec_temp_list = []
    packing_spec_title = product_name.lower().replace(" ","")
    for i in range(len(common_unit_list)):
        packing_spec_title = packing_spec_title.replace(common_unit_list_zn[i], common_unit_list[i])
    for i in soup.find_all("tr", class_="productPackingSpec"):
        if "Packing Spec" in i.text:
            for j in i.find_all("td"):
                if j.text != "Packing Spec":
                    packing_spec = j.text.lower().replace(",","").replace(" ","")
                    for k in range(len(common_unit_list)):
                        packing_spec = packing_spec.replace(common_unit_list_zn[k], common_unit_list[k])
        if "Country of Origin" in i.text:
            for j in i.find_all("span"):
                if j.text != "Country of Origin":
                    country_of_origin = j.text

    for unit in common_unit_list:
        if re.search(f"\d+{unit}", packing_spec_title) != None:
            packing_spec_temp_list.append(re.search(f"\d+{unit}", packing_spec_title).group())
        if re.search(f"\d+{unit}", packing_spec) != None:
            packing_spec_temp_list.append(re.search(f"\d+{unit}", packing_spec).group())
    packing_spec_temp_list = list(set(packing_spec_temp_list))
    for unit in common_unit_list:
        for record in packing_spec_temp_list:
            try:
                packing_spec_number_list.append(re.search(f"\d+{unit}", record).group().split(unit)[0])
                packing_spec_unit_list.append(unit)
            except:
                continue
    del(packing_spec)

    remark = soup.find("tr", class_="productRemark").find("span", class_="short-desc").text

    ## Part 4)
    try:
        description_video = len(soup.find("div", id="videoTab").find("ul", class_="video-list").find_all("li"))
    except:
        description_video = 0
    try:
        description_img_list = [temp_url["src"] for temp_url in soup.find("div", id="photoTab").find_all("img")]
    except:
        description_img_list = []
    description = soup.find("div", id="descriptionsTab").find("div", class_="tabBody").text.replace("\n", "")

    ## Part 5)
    url += "?scrollTo=reviewTab"
    driver.get(url)
    time.sleep(1)
    try:
        driver.find_element(By.CLASS_NAME, "btnCloseLarge").click()
    except:
        pass
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    if soup.find("div", class_="noReviews")["style"] != "":
        customer_id = []
        customer_name = []
        customer_class = []
        customer_rate = []
        customer_review_date = []
        customer_comment = []
        customer_recommend = []
        
        next_page_checking = soup.find("a", class_="next-btn")["class"]
        while "disable" not in next_page_checking:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            next_page_checking = soup.find("a", class_="next-btn")["class"]
            
            for i in soup.find_all("div", class_="product-review-wrapper"):
                customer_id.append(i.a.attrs["data-user"])
                customer_name.append(i.a.text)
                if "vip" in i.find_all("span")[1].attrs["class"][0]:
                    customer_class.append(i.find_all("span")[1].attrs["class"][0].replace("-highlight",""))
                else:
                    customer_class.append("normal")
                customer_rate.append(len(i.find_all("div", class_="star-container")[1]))
                customer_review_date.append(i.find("span", class_="review-date").text)
                customer_comment.append(i.find("div", class_="review-title").text)
                try:
                    if "not" not in i.find("span", class_="recommendOrNot").text:
                        customer_recommend.append(1)
                    else:
                        customer_recommend.append(0)
                except:
                    customer_recommend.append(np.nan)
        
            next_page_button = driver.find_element(By.CLASS_NAME, "next-btn")
            next_page_button.location_once_scrolled_into_view
            action = ActionChains(driver)
            action.key_down(Keys.ARROW_UP).key_up(Keys.ARROW_UP).perform()
            next_page_button.click()
            time.sleep(3)

In [ ]:
# https://brandirectory.com/search/{L'Oreal%20Paris} L'Oreal Paris